## 🤗 Finetune **Longformer Encoder-Decoder (LED)** on 8K Tokens 🤗

The *Longformer Encoder-Decoder (LED)* was recently added as an extension to [Longformer: The Long-Document Transformer](https://arxiv.org/abs/2004.05150) by Iz Beltagy, Matthew E. Peters, Arman Cohan.

In this notebook we will finetune *LED* for Summarization on [Pubmed](https://huggingface.co/datasets/viewer/?dataset=scientific_papers). *Pubmed* is a long-range summarization dataset, which makes it a good candidate for LED. LED will be finetuned up to an input length of 8K tokens on a single GPU.

We will leverage 🤗`Seq2SeqTrainer`, gradient checkpointing and as usual 🤗`datasets`.

First, let's try to get a GPU with at least 15GB RAM.

In [ ]:
# crash colab to get more RAM
# !kill -9 -1

To check that we are having enough RAM we can run the following command.
If the randomely allocated GPU is too small, the above cells can be run
to crash the notebook hoping to get a better GPU.

Next, we install 🤗Transformers, 🤗Datasets, and `rouge_score`.



In [ ]:
%%capture
!pip install datasets
!pip install transformers
!pip install rouge_score

Let's start by loading and preprocessing the dataset.



In [ ]:
from datasets import load_dataset, load_metric

Next, we download the pubmed train and validation dataset ([click to see on 🤗Datasets Hub](https://huggingface.co/datasets/scientific_papers)). This can take a couple of minutes **☕** .

In [ ]:

tr_dataset = load_dataset("scientific_papers", "arxiv", split="train")
train_dataset = tr_dataset.select(range(400))

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/203037 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6436 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [ ]:
v_dataset = load_dataset("scientific_papers", "arxiv", split="validation")
val_dataset=v_dataset.select(range(100))

It's always a good idea to take a look at some data samples. Let's do that here.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=4):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(val_dataset)

We can see that the input data is the `article` - a scientific report and the target data is the `abstract` - a concise summary of the report.

Cool! Having downloaded the dataset, let's tokenize it.
We'll import the convenient `AutoTokenizer` class.

In [ ]:
from transformers import AutoTokenizer

 and load the tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")

Note that for the sake of this notebook, we finetune the "smaller" LED checkpoint ["allenai/led-base-16384"](https://huggingface.co/allenai/led-base-16384). Better performance can however be attained by finetuning ["allenai/led-large-16384"](https://huggingface.co/allenai/led-large-16384) at the cost of a higher required GPU RAM.

Pubmed's input data has a median token length of 2715 with the 90%-ile token length being 6101. The output data has a media token length of 171 with the 90%-ile token length being 352.${}^1$.

Thus, we set the maximum input length to 8192 and the maximum output length to 512 to ensure that the model can attend to almost all input tokens is able to generate up to a large enough number of output tokens.

In this notebook, we are only able to train on `batch_size=2` to prevent out-of-memory errors.

---
${}^1$ The data is taken from page 11 of [Big Bird: Transformers for Longer Sequences](https://arxiv.org/pdf/2007.14062.pdf).


In [ ]:
max_input_length = 8192
max_output_length = 512
batch_size = 2

Now, let's write down the input data processing function that will be used to map each data sample to the correct model format.
As explained earlier `article` represents here our input data and `abstract` is the target data. The datasamples are thus tokenized up to the respective maximum lengths of 8192 and 512.

In addition to the usual `attention_mask`, LED can make use of an additional `global_attention_mask` defining which input tokens are attended globally and which are attended only locally, just as it's the case of [Longformer](https://huggingface.co/transformers/model_doc/longformer.html). For more information on Longformer's self-attention, please take a look at the corresponding [docs](https://huggingface.co/transformers/model_doc/longformer.html#longformer-self-attention). For summarization, we follow recommendations of the [paper](https://arxiv.org/abs/2004.05150) and use global attention only for the very first token. Finally, we make sure that no loss is computed on padded tokens by setting their index to `-100`.

In [ ]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["article"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["abstract"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

Great, having defined the mapping function, let's preprocess the training data

In [ ]:
train_dataset = train_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract", "section_names"],
)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

and validation data

In [ ]:
val_dataset = val_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract", "section_names"],
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Finally, the datasets should be converted into the PyTorch format as follows.

In [ ]:
train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)
val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

Alright, we're almost ready to start training. Let's load the model via the `AutoModelForSeq2SeqLM` class.

In [ ]:
from transformers import AutoModelForSeq2SeqLM

We've decided to stick to the smaller model `"allenai/led-base-16384"` for the sake of this notebook. In addition, we directly enable gradient checkpointing and disable the caching mechanism to save memory.

In [ ]:
led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384", gradient_checkpointing=True, use_cache=False)

During training, we want to evaluate the model on Rouge, the most common metric used in summarization, to make sure the model is indeed improving during training. For this, we set fitting generation parameters. We'll use beam search with a small beam of just 2 to save memory. Also, we force the model to generate at least 100 tokens, but no more than 512. In addition, some other generation parameters are set that have been found helpful for generation. For more information on those parameters, please take a look at the [docs](https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.generation_utils.GenerationMixin.generate).

In [ ]:
# set generate hyperparameters
led.config.num_beams = 2
led.config.max_length = 512
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

Next, we also have to define the function the will compute the `"rouge"` score during evalution.

Let's load the `"rouge"` metric from 🤗datasets and define the `compute_metrics(...)` function.

In [ ]:
rouge = load_metric("rouge")

<ipython-input-21-6f1857dc142f>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


The compute metrics function expects the generation output, called `pred.predictions` as well as the gold label, called `pred.label_ids`.

Those tokens are decoded and consequently, the rouge score can be computed.

In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"]
    )["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

Now, we're ready to start training. Let's import the `Seq2SeqTrainer` and `Seq2SeqTrainingArguments`.

In contrast to the usual `Trainer`, the `Seq2SeqTrainer` makes it possible to use the `generate()` function during evaluation. This should be enabled with `predict_with_generate=True`. Because our GPU RAM is limited, we make use of gradient accumulation by setting `gradient_accumulation_steps=4` to have an effective `batch_size` of 2 * 4 = 8.

Other training arguments can be read upon in the [docs](https://huggingface.co/transformers/main_classes/trainer.html?highlight=trainingarguments#transformers.TrainingArguments).

In [ ]:
!pip install accelerate -U



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.6 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]

In [ ]:
! pip install -U accelerate
! pip install -U transformers

In [ ]:
import accelerate
import transformers
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer


transformers.__version__, accelerate.__version__

('4.31.0', '0.21.0')

In [ ]:
! pip install -U accelerate
! pip install -U transformers

In [ ]:
# enable fp16 apex training
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    output_dir="./",
    logging_steps=5,
    eval_steps=10,
    save_steps=10,
    save_total_limit=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
)

NameError: ignored

In [ ]:
trainer = Seq2SeqTrainer(
    model=led,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

NameError: ignored

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LEDTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,3.190000,3.133845,0.130100,0.084800,0.097800
20,3.169500,2.928536,0.088700,0.136400,0.100200
30,2.767600,2.813875,0.142600,0.096000,0.109600


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


TrainOutput(global_step=37, training_loss=3.161574634345802, metrics={'train_runtime': 2867.3299, 'train_samples_per_second': 0.105, 'train_steps_per_second': 0.013, 'total_flos': 1598521262211072.0, 'train_loss': 3.161574634345802, 'epoch': 0.99})

In [ ]:
import torch

# Save the model's state dictionary
torch.save(led.state_dict(), "/content/drive/project.pt")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
a_test = load_dataset("scientific_papers", "arxiv", ignore_verifications=True, split="test")
arxiv_test = a_test.select(range(5))
def generate_answer(batch):
  inputs_dict = tokenizer(batch["article"], padding="max_length", max_length=8192, return_tensors="pt", truncation=True)
  input_ids = inputs_dict.input_ids.to("cuda")
  attention_mask = inputs_dict.attention_mask.to("cuda")
  global_attention_mask = torch.zeros_like(attention_mask)
  # put global attention on <s> token
  global_attention_mask[:, 0] = 1

  predicted_abstract_ids = led.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)
  batch["predicted_abstract"] = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)
  return batch


result = arxiv_test.map(generate_answer, batched=True, batch_size=1)
# load rouge
rouge = load_metric("rouge")

rouge_types = ["rouge1", "rouge2", "rougeL"]

rouge_scores = rouge.compute(predictions=result["predicted_abstract"], references=result["abstract"], rouge_types=rouge_types)

for rouge_type in rouge_types:
    print(f"Rouge-{rouge_type} scores:")
    print("F-score:", rouge_scores[rouge_type].mid.fmeasure)
    print("Precision:", rouge_scores[rouge_type].mid.precision)
    print("Recall:", rouge_scores[rouge_type].mid.recall)
    print()


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Rouge-rouge1 scores:
F-score: 0.4323790276846935
Precision: 0.47399344569288393
Recall: 0.4141992220041001

Rouge-rouge2 scores:
F-score: 0.1949194768632888
Precision: 0.21429141418078818
Recall: 0.18324093292129823

Rouge-rougeL scores:
F-score: 0.2668515824458856
Precision: 0.2882063780568407
Recall: 0.2565145764091713



In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.1 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import LEDForConditionalGeneration, LEDTokenizer

# Load the tokenizer
tokenizer = LEDTokenizer.from_pretrained("allenai/led-base-16384")

# Create the model architecture and move it to the GPU
led = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384").to("cuda")

# Load the model's state dictionary
led.load_state_dict(torch.load("/content/drive/MyDrive/project.pt", map_location="cuda"))
def generate_output(input_text, max_length=1024, min_length=50):
    input_ids = tokenizer(input_text, max_length=8192, return_tensors="pt", truncation=True).input_ids.to("cuda")
    attention_mask = torch.ones_like(input_ids).to("cuda")
    global_attention_mask = torch.zeros_like(attention_mask)
    # put global attention on <s> token
    global_attention_mask[:, 0] = 1

    predicted_abstract_ids = led.generate(
        input_ids,
        attention_mask=attention_mask,
        global_attention_mask=global_attention_mask,
        max_length=max_length,
        min_length=min_length,
    )
    predicted_abstract = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)
    return predicted_abstract[0]


input_text = "Flow matching is a recent framework to train generative models that exhibits impressive empirical performance while being relatively easier to train compared with diffusion-based models. Despite its advantageous properties, prior methods still face the challenges of expensive computing and a large number of function evaluations of off-the-shelf solvers in the pixel space. Furthermore, although latentbased generative methods have shown great success in recent years, this particular model type remains underexplored in this area. In this work, we propose to apply flow matching in the latent spaces of pretrained autoencoders, which offers improved computational efficiency and scalability for high-resolution image synthesis. This enables flow-matching training on constrained computational resources while maintaining their quality and flexibility. Additionally, our work stands as a pioneering contribution in the integration of various conditions into flow matching for conditional generation tasks, including label-conditioned image generation, image inpainting, and semantic-to-image generation. Through extensive experiments, our approach demonstrates its effectiveness in both quantitative and qualitative results on various datasets, such as CelebA-HQ, FFHQ, LSUN Church & Bedroom, and ImageNet. We also provide a theoretical control of the Wasserstein-2 distance between the reconstructed latent flow distribution and true data distribution, showing it is upper-bounded by the latent flow matching objective."
max_length = 150  # Set the maximum length for the generated output
min_length = 20   # Set the minimum length for the generated output
output_text = generate_output(input_text, max_length=max_length, min_length=min_length)
print("Generated output:")
print(output_text)


Generated output:
Flow matching is a recent framework to train generative models that exhibits impressive empirical performance while being relatively easier to train compared with diffusion-based models. Despite its advantageous properties, prior methods still face the challenges of expensive computing and a large number of function evaluations of off-the-shelf solvers in the pixel space. Furthermore, although latentbased generative methods have shown great success in recent years, this particular model type remains underexplored in this area. In this work, we propose to apply flow matching in the latent spaces of pretrained autoencoders, which offers improved computational efficiency and scalability for high-resolution image synthesis. This enables flow-matching training on constrained computational resources while maintaining their quality and flexibility


In [ ]:
from transformers import pipeline
summarizer = pipeline(task="summarization", model="facebook/bart-large-cnn")

input_text = """
Flow matching is a recent framework to train generative models that exhibits impressive empirical performance while being relatively easier to train compared with diffusion-based models. Despite its advantageous properties, prior methods still face the challenges of expensive computing and a large number of function evaluations of off-the-shelf solvers in the pixel space. Furthermore, although latentbased generative methods have shown great success in recent years, this particular model type remains underexplored in this area. In this work, we propose to apply flow matching in the latent spaces of pretrained autoencoders, which offers improved computational efficiency and scalability for high-resolution image synthesis. This enables flow-matching training on constrained computational resources while maintaining their quality and flexibility. Additionally, our work stands as a pioneering contribution in the integration of various conditions into flow matching for conditional generation tasks, including label-conditioned image generation, image inpainting, and semantic-to-image generation. Through extensive experiments, our approach demonstrates its effectiveness in both quantitative and qualitative results on various datasets, such as CelebA-HQ, FFHQ, LSUN Church & Bedroom, and ImageNet. We also provide a theoretical control of the Wasserstein-2 distance between the reconstructed latent flow distribution and true data distribution, showing it is upper-bounded by the latent flow matching objective.
"""

summary = summarizer(input_text, max_length=150, min_length=20, do_sample=True)[0]['summary_text']
print(summary)


Flow matching is a framework to train generative models. It offers improved computational efficiency and scalability for high-resolution image synthesis. We propose to apply it in the latent spaces of pretrained autoencoders.


#NOT IMPORTANT

In [ ]:
import torch
from transformers import LEDForConditionalGeneration, LEDTokenizer

# Load the tokenizer
tokenizer = LEDTokenizer.from_pretrained("allenai/led-base-16384")

# Create the model architecture and move it to the GPU
led = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384").to("cuda")

# Load the model's state dictionary
led.load_state_dict(torch.load("/content/drive/MyDrive/project.pt", map_location="cuda"))
def generate_output(input_text, max_length=1024, min_length=50):
    input_ids = tokenizer(input_text, max_length=8192, return_tensors="pt", truncation=True).input_ids.to("cuda")
    attention_mask = torch.ones_like(input_ids).to("cuda")
    global_attention_mask = torch.zeros_like(attention_mask)
    # put global attention on <s> token
    global_attention_mask[:, 0] = 1

    predicted_abstract_ids = led.generate(
        input_ids,
        attention_mask=attention_mask,
        global_attention_mask=global_attention_mask,
        max_length=max_length,
        min_length=min_length,
    )
    predicted_abstract = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)
    return predicted_abstract[0]


input_text = "nAs the urgency to reduce greenhouse gas emissions grows, Microgrids(MGs) are emerging as an effective and efficient solution for integrating distributed renewable resources [1]. Compared to centralized power grids, MGs leverage localized generation, minimizing transmission losses and eliminating the need for significant infrastructure adjustments. Another noteworthy advantage of MGs is their ability to enhance grid resilience by operating in “islanded mode,” enabling selfsustainability during outages [2]. Furthermore, MGs play a crucial role in electrifying rural areas, leading to a substantial surge in their numbers [3].\nThe energy management of a MG, involving distributed generation units (both conventional and renewable) and energy storage units, requires making sequential decisions in the face of uncertainties introduced by renewable energy sources. The optimal dispatch for MG energy management under these\nM. V. Liu is with the Field of Systems Engineering, Cornell University, Ithaca, NY, 14853, USA (e-mail: ml2589@cornell.edu)\nP. M. Reed and D. Gold are with Civil and Environmental Engineering, Cornell University, Ithaca, NY, 14853, USA (e-mail: patrick.reed@cornell.edu; dfg42@cornell.edu).\nG. Quist is with the Facilities and Campus Service, Cornell University, Ithaca, NY, 14853, USA (e-mail: gq29@cornell.edu).\nC. L. Anderson is with the Field of Systems Engineering and Cornell Energy Systems Institute, Cornell University, Ithaca, NY, 14853, USA (email: cla28@cornell.edu)\nuncertainties is often addressed by representing the control problem as a Markov Decision Process (MDP) [4], [5]. The MDP represents a sequential decision process in which an agent interacts with an environment, making decisions at each time step to maximize its long-term expected rewards [6]. Traditionally, the MDP control formulations have been tackled using the Dynamic Programming (DP) methods, which are often burdened by the “curse of dimensionality,” limiting the size of the state and action spaces that can be addressed [7]. Moreover, traditional MDP formulations require knowledge of the “model”, which refers to the transition probability and reward function for given state-action pairs for the system of consideration. Unfortunately, in energy systems these transition probabilities are often not known due to the complexity and non-linearity of the systems, prompting the need for a “model-free” approach.\nTo overcome these issues, Reinforcement Learning (RL) has gained popularity in power system operation and control due to its model-free nature [8]. RL, also known as model-free approximate dynamic programming (ADP), has been combined with artificial neural networks (ANNs) for function approximations, leading to the development of Deep RL (DRL) methods [9]. Numerous advancements have been made in employing (D)RL algorithms to enhance demandside energy management [7], [10], [11]. For a comprehensive overview of RL in the context of power system operations, interested readers can refer to [12]. A common practice of these studies is to take advantage of the data-driven property of RL and use historical data to train the agent by simulating its interaction with the environment without oversimplifying the state and action spaces. However, prior studies have primarily focused on a single objective, typically maximizing profit (minimizing cost), due to the inherent nature of the underlying Bellman equations formulation. In fact, methods focused on multiple objectives are exceedingly rare in the RL and ADP bodies of literature [13], [14]. As the share of renewable energy increases in MGs, other objectives, such as environmental impacts, operation reliability, and effective storage operation, are receiving more attention [15]. A sole focus on a single objective could lead to control solutions in the extreme corners of the broader space of relevant performance objectives and fails to properly represent the interests of stakeholders, especially when conflicts exist between their objective functions [16], [17].\nIn addressing multiple objectives in MG optimization, a standard approach is to employ a weighted sum method to convert the original multiple objective formulations into a single objective representation that tacitly infers that the specified\nar X\niv :2\n30 7.\n08 69\n2v 1\n[ ee\nss .S\nY ]\n1 7\nJu l 2\n02 3\n2 weights capture all stakeholders’ preferences. The weighted single objective is the dominant approach for handling multiple objectives in DP formulations [18]. If a user is interested in an explicit representation of the full suite of objective tradeoffs, solving repeated weighted DP instances can become computationally prohibitive. Identifying all of the Pareto optimal solutions1 that compose control tradeoffs becomes increasingly challenging as the number of objectives grows, leading to a factorial growth in the computation cost [19]. In [20]–[23], the weighted sum approach is combined with fuzzy techniques to limit the number of combinations of weights, reducing the complexity of the computation. However, these methods struggle to effectively explore the tradeoffs between the objectives. The predefined preferences for objectives can potentially overlook superior solutions, particularly in scenarios where system dynamics are non-convex and non-linear [24]. This limitation hampers the ability to find optimal solutions that strike the best balance among conflicting objectives.\nMeta-heuristic methods such as the Non-dominated Sorting Genetic Algorithm (NSGA-II) [25] and Multiple Objective Particle Swarm Optimization (MOPSO) [26] have been employed in optimizing MG operation to allow for an explicit search of the Pareto-optimal solutions [27], [28]. These multiobjective optimization algorithms use population sorting techniques to guide search and maintain a set of non-dominated solutions, resulting in a diverse set of Pareto-optimal solutions for decision-makers without specifying predefined preferences for objectives. In addition, as simulation-based optimization, the meta-heuristic methods can handle more complex system formulations that are non-linear and non-convex, making them a good candidate class of solution tools for the demand-side energy management problem [29].\nIn this paper, we present a novel framework that combines the strengths of multi-objective optimization and RL to tackle the energy management problem of MGs. We use the Borg Multi-Objective Evolutionary Algorithm (MOEA) [30], which has been proven to meet or exceed the performance of other MOEAs in complex system planning and operation applications [31]–[33], explicitly exploring their tradeoffs in the higher-dimensional objective spaces. Our framework proposes a model-free policy approximation approach, enabling the agent to interact with the unknown environment in continuous state and action spaces while managing the computation complexity of the stochastic MG control problem of focus. The proposed Multi-Objective Reinforcement Learning (MORL) framework is applied to the challenging abstraction of the Combined Heat and Power (CHP) CU-MG, demonstrating its efficacy in a real-world application. Using historical weather, demand, and utility price data, the trained agent (policy) makes collaborative decisions for multiple energy sources under stochastic state conditions. This model-free policy approximation approach not only reduces computational complexity but also provides interpretability in the trained parametric policy, allowing us to understand how the agent utilizes exogenous information to shape its dynamic and adaptive control actions.\n1Pareto-optimal solution refers to a solution where no objective can be improved without sacrificing another objective.\nThe primary contributions of this study are as follows: • a novel framework that combines the power of multi-\nobjective optimization and RL to support MG energy management under uncertainty, • a data-driven model-free policy approximation that reduces computation complexity while providing interpretability to the policy trained under multiple conflicting objectives, and • demonstrated performance on an existing CHP MG to highlight the tradeoffs between multiple objectives and the importance of exogenous information in improving stochastic decisions relative to current operations.\nThe rest of this paper is organized as follows. Section II introduces the real-world CU-MG and the system modeling formulations. Section III presents the proposed MORL framework and formulates the multi-objective energy management problem under it. Section IV demonstrates the numerical results for the test case. Section V concludes this paper."

max_length = 200  # Set the maximum length for the generated output
min_length = 20   # Set the minimum length for the generated output
output_text = generate_output(input_text, max_length=max_length, min_length=min_length)
print("Generated output:")
print(output_text)


Generated output:
The power system operations of the Combined Heat and Power (CHP) CU-MG are being studied under uncertainty. The problem of MG energy management is often addressed by using a multi-objective optimization approach to solve the problem of multiple objectives. The problem of MG energy management is often addressed by using a model-free policy approximation approach to the problem of multiple objectives. However, the problem of MG energy management is often addressed by using a model-free policy approximation approach to the problem of multiple objectives. The problem of MG energy management is often addressed by using a multi-objective optimization approach to solve the problem of multiple objectives. The problem of MG energy management is often addressed by using a multi-objective optimization approach to solve the problem of multiple objectives [1]. The problem of MG energy management is often addressed by using a multi-objective optimization approach to the problem of 

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

def generate_summary(input_text, max_length=512):
    input_text = input_text.strip()
    tokenized_text = tokenizer.encode(input_text, add_special_tokens=True, max_length=max_length, truncation=True, return_tensors="pt")
    summary_ids = model.generate(tokenized_text, num_beams=4, max_length=200, min_length=20, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

input_text = """
    As the urgency to reduce greenhouse gas emissions grows, Microgrids(MGs) are emerging as an effective and efficient solution for integrating distributed renewable resources [1]. Compared to centralized power grids, MGs leverage localized generation, minimizing transmission losses and eliminating the need for significant infrastructure adjustments. Another noteworthy advantage of MGs is their ability to enhance grid resilience by operating in “islanded mode,” enabling selfsustainability during outages [2]. Furthermore, MGs play a crucial role in electrifying rural areas, leading to a substantial surge in their numbers [3].\nThe energy management of a MG, involving distributed generation units (both conventional and renewable) and energy storage units, requires making sequential decisions in the face of uncertainties introduced by renewable energy sources. The optimal dispatch for MG energy management under these\nM. V. Liu is with the Field of Systems Engineering, Cornell University, Ithaca, NY, 14853, USA (e-mail: ml2589@cornell.edu)\nP. M. Reed and D. Gold are with Civil and Environmental Engineering, Cornell University, Ithaca, NY, 14853, USA (e-mail: patrick.reed@cornell.edu; dfg42@cornell.edu).\nG. Quist is with the Facilities and Campus Service, Cornell University, Ithaca, NY, 14853, USA (e-mail: gq29@cornell.edu).\nC. L. Anderson is with the Field of Systems Engineering and Cornell Energy Systems Institute, Cornell University, Ithaca, NY, 14853, USA (email: cla28@cornell.edu)\nuncertainties is often addressed by representing the control problem as a Markov Decision Process (MDP) [4], [5]. The MDP represents a sequential decision process in which an agent interacts with an environment, making decisions at each time step to maximize its long-term expected rewards [6]. Traditionally, the MDP control formulations have been tackled using the Dynamic Programming (DP) methods, which are often burdened by the “curse of dimensionality,” limiting the size of the state and action spaces that can be addressed [7]. Moreover, traditional MDP formulations require knowledge of the “model”, which refers to the transition probability and reward function for given state-action pairs for the system of consideration. Unfortunately, in energy systems these transition probabilities are often not known due to the complexity and non-linearity of the systems, prompting the need for a “model-free” approach.\nTo overcome these issues, Reinforcement Learning (RL) has gained popularity in power system operation and control due to its model-free nature [8]. RL, also known as model-free approximate dynamic programming (ADP), has been combined with artificial neural networks (ANNs) for function approximations, leading to the development of Deep RL (DRL) methods [9]. Numerous advancements have been made in employing (D)RL algorithms to enhance demandside energy management [7], [10], [11]. For a comprehensive overview of RL in the context of power system operations, interested readers can refer to [12]. A common practice of these studies is to take advantage of the data-driven property of RL and use historical data to train the agent by simulating its interaction with the environment without oversimplifying the state and action spaces. However, prior studies have primarily focused on a single objective, typically maximizing profit (minimizing cost), due to the inherent nature of the underlying Bellman equations formulation. In fact, methods focused on multiple objectives are exceedingly rare in the RL and ADP bodies of literature [13], [14]. As the share of renewable energy increases in MGs, other objectives, such as environmental impacts, operation reliability, and effective storage operation, are receiving more attention [15]. A sole focus on a single objective could lead to control solutions in the extreme corners of the broader space of relevant performance objectives and fails to properly represent the interests of stakeholders, especially when conflicts exist between their objective functions [16], [17].\nIn addressing multiple objectives in MG optimization, a standard approach is to employ a weighted sum method to convert the original multiple objective formulations into a single objective representation that tacitly infers that the specified\nar X\niv :2\n30 7.\n08 69\n2v 1\n[ ee\nss .S\nY ]\n1 7\nJu l 2\n02 3\n2 weights capture all stakeholders’ preferences. The weighted single objective is the dominant approach for handling multiple objectives in DP formulations [18]. If a user is interested in an explicit representation of the full suite of objective tradeoffs, solving repeated weighted DP instances can become computationally prohibitive. Identifying all of the Pareto optimal solutions1 that compose control tradeoffs becomes increasingly challenging as the number of objectives grows, leading to a factorial growth in the computation cost [19]. In [20]–[23], the weighted sum approach is combined with fuzzy techniques to limit the number of combinations of weights, reducing the complexity of the computation. However, these methods struggle to effectively explore the tradeoffs between the objectives. The predefined preferences for objectives can potentially overlook superior solutions, particularly in scenarios where system dynamics are non-convex and non-linear [24]. This limitation hampers the ability to find optimal solutions that strike the best balance among conflicting objectives.\nMeta-heuristic methods such as the Non-dominated Sorting Genetic Algorithm (NSGA-II) [25] and Multiple Objective Particle Swarm Optimization (MOPSO) [26] have been employed in optimizing MG operation to allow for an explicit search of the Pareto-optimal solutions [27], [28]. These multiobjective optimization algorithms use population sorting techniques to guide search and maintain a set of non-dominated solutions, resulting in a diverse set of Pareto-optimal solutions for decision-makers without specifying predefined preferences for objectives. In addition, as simulation-based optimization, the meta-heuristic methods can handle more complex system formulations that are non-linear and non-convex, making them a good candidate class of solution tools for the demand-side energy management problem [29].\nIn this paper, we present a novel framework that combines the strengths of multi-objective optimization and RL to tackle the energy management problem of MGs. We use the Borg Multi-Objective Evolutionary Algorithm (MOEA) [30], which has been proven to meet or exceed the performance of other MOEAs in complex system planning and operation applications [31]–[33], explicitly exploring their tradeoffs in the higher-dimensional objective spaces. Our framework proposes a model-free policy approximation approach, enabling the agent to interact with the unknown environment in continuous state and action spaces while managing the computation complexity of the stochastic MG control problem of focus. The proposed Multi-Objective Reinforcement Learning (MORL) framework is applied to the challenging abstraction of the Combined Heat and Power (CHP) CU-MG, demonstrating its efficacy in a real-world application. Using historical weather, demand, and utility price data, the trained agent (policy) makes collaborative decisions for multiple energy sources under stochastic state conditions. This model-free policy approximation approach not only reduces computational complexity but also provides interpretability in the trained parametric policy, allowing us to understand how the agent utilizes exogenous information to shape its dynamic and adaptive control actions.\n1Pareto-optimal solution refers to a solution where no objective can be improved without sacrificing another objective.\nThe primary contributions of this study are as follows: • a novel framework that combines the power of multi-\nobjective optimization and RL to support MG energy management under uncertainty, • a data-driven model-free policy approximation that reduces computation complexity while providing interpretability to the policy trained under multiple conflicting objectives, and • demonstrated performance on an existing CHP MG to highlight the tradeoffs between multiple objectives and the importance of exogenous information in improving stochastic decisions relative to current operations.\nThe rest of this paper is organized as follows. Section II introduces the real-world CU-MG and the system modeling formulations. Section III presents the proposed MORL framework and formulates the multi-objective energy management problem under it. Section IV demonstrates the numerical results for the test case. Section V concludes this paper.
"""

# Split the text into smaller segments and generate summaries for each segment
segment_size = 512
segments = [input_text[i:i+segment_size] for i in range(0, len(input_text), segment_size)]
generated_summaries = [generate_summary(segment) for segment in segments]

# Combine the generated summaries into a final summary
final_summary = " ".join(generated_summaries)
print(final_summary)


As the urgency to reduce greenhouse gas emissions grows, Microgrids(MGs) are emerging as an effective and efficient solution for integrating distributed renewable resources. Compared to centralized power grids, MGs leverage localized generation, minimizing transmission losses and eliminating the need for significant infrastructure adjustments. Another noteworthy advantage of MGs is their ability to enhance grid resilience by operating in “islanded mode” The energy management of a MG, involving distributed generation units (both conventional and renewable) and energy storage units, requires making sequential decisions in the face of uncertainties introduced by renewable energy sources. MGs play a crucial role in electrifying rural areas, leading to a substantial surge in their numbers. (e-mail: ml2589@cornell.edu) P. Reed and D. Gold are with Civil and Environmental Engineering, Cornell University, Ithaca, NY, 14853, USA. G. Quist is with the Facilities and Campus Service. L. Anderson i

## Training

In [ ]:
#!/usr/bin/env python3
from datasets import load_dataset, load_metric
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
)

# load rouge
rouge = load_metric("rouge")

# load pubmed
pubmed_train = load_dataset("scientific_papers", "pubmed", ignore_verifications=True, split="train")
pubmed_val = load_dataset("scientific_papers", "pubmed", ignore_verifications=True, split="validation[:10%]")

# comment out following lines for a test run
# pubmed_train = pubmed_train.select(range(32))
# pubmed_val = pubmed_val.select(range(32))

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/led-large-16384")


# max encoder length is 8192 for PubMed
encoder_max_length = 8192
decoder_max_length = 512
batch_size = 2


def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["article"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
    )
    outputs = tokenizer(
        batch["abstract"],
        padding="max_length",
        truncation=True,
        max_length=decoder_max_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch


# map train data
pubmed_train = pubmed_train.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract", "section_names"],
)

# map val data
pubmed_val = pubmed_val.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract", "section_names"],
)

# set Python list to PyTorch tensor
pubmed_train.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

# set Python list to PyTorch tensor
pubmed_val.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

# enable fp16 apex training
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    fp16_backend="apex",
    output_dir="./",
    logging_steps=250,
    eval_steps=5000,
    save_steps=500,
    warmup_steps=1500,
    save_total_limit=2,
    gradient_accumulation_steps=4,
)


# compute Rouge score during validation
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"]
    )["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }


# load model + enable gradient checkpointing & disable cache for checkpointing
led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-large-16384", gradient_checkpointing=True, use_cache=False)

# set generate hyperparameters
led.config.num_beams = 4
led.config.max_length = 512
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3


# instantiate trainer
trainer = Seq2SeqTrainer(
    model=led,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=pubmed_train,
    eval_dataset=pubmed_val,
)

# start training
trainer.train()

## Evaluation

In [ ]:
import torch

from datasets import load_dataset, load_metric
from transformers import LEDTokenizer, LEDForConditionalGeneration

# load pubmed
a_test = load_dataset("scientific_papers", "arxiv", ignore_verifications=True, split="test")
arxiv_test = a_dataset.select(range(25))
# load tokenizer
tokenizer = LEDTokenizer.from_pretrained("patrickvonplaten/led-large-16384-pubmed")
model = LEDForConditionalGeneration.from_pretrained("patrickvonplaten/led-large-16384-pubmed").to("cuda").half()


def generate_answer(batch):
  inputs_dict = tokenizer(batch["article"], padding="max_length", max_length=8192, return_tensors="pt", truncation=True)
  input_ids = inputs_dict.input_ids.to("cuda")
  attention_mask = inputs_dict.attention_mask.to("cuda")
  global_attention_mask = torch.zeros_like(attention_mask)
  # put global attention on <s> token
  global_attention_mask[:, 0] = 1

  predicted_abstract_ids = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)
  batch["predicted_abstract"] = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)
  return batch


result = pubmed_test.map(generate_answer, batched=True, batch_size=4)

# load rouge
rouge = load_metric("rouge")

print("Result:", rouqge.compute(predictions=result["predicted_abstract"], references=result["abstract"], rouge_types=["rouge2"])["rouge2"].mid)

In [ ]:
from transformers import AutoTokenizer
import torch
import itertools
tokenizer = AutoTokenizer.from_pretrained('google/pegasus-wikihow')

for epoch in range(1):
    for batch in train_loader:

            batch['article'] = [tokenizer.encode(text, padding='max_length', truncation=True, max_length=512) for text in batch['article']]

            batch['attention_mask'] = [[1] * len(article) + [0] * (512 - len(article)) for article in batch['article']]

            batch['decoder_input_ids'] = [b.tolist() for b in batch['decoder_input_ids']]
            batch['decoder_attention_mask'] = [b.tolist() for b in batch['decoder_attention_mask']]

            batch['labels'] = [b.tolist() for b in batch['labels']]

            batch = {
                'input_ids': torch.tensor(batch['article']).to(device),
                'attention_mask': torch.tensor(batch['attention_mask']).to(device),
                'decoder_input_ids': torch.tensor(batch['decoder_input_ids']).to(device),
                'decoder_attention_mask': torch.tensor(batch['decoder_attention_mask']).to(device),
                'labels': torch.tensor(batch['labels']).to(device)

               }

            inputs = {
                "input_ids": batch['input_ids'],
                "attention_mask": batch['attention_mask'],
                "decoder_input_ids": batch['decoder_input_ids'],
                "decoder_attention_mask": batch['decoder_attention_mask'],
                "labels": batch['labels'],
            }

            optimizer.zero_grad()

            outputs = model(**inputs)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()




In [ ]:
model.eval()

generated_summaries = []
target_summaries = []

with torch.no_grad():
    for batch in test_loader:
        inputs = {
            "input_ids": torch.stack(batch["input_ids"]).to(device),
            "attention_mask": torch.stack(batch["attention_mask"]).to(device),
            "decoder_input_ids": torch.stack(batch["decoder_input_ids"]).to(device),
            "decoder_attention_mask": torch.stack(batch["decoder_attention_mask"]).to(device),
            "labels": torch.stack(batch["labels"]).to(device),
        }

        outputs = model.generate(
            input_ids=inputs["decoder_input_ids"],
            attention_mask=inputs["decoder_attention_mask"],
            max_length=128,
            num_beams=4,
            num_return_sequences=1,
            early_stopping=True,
        )
        generated_summaries.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))
        target_summaries.extend(tokenizer.batch_decode(inputs["labels"], skip_special_tokens=True))

# Print the generated and target summaries
for generated_summary, target_summary in zip(generated_summaries, target_summaries):
    print("Generated Summary:", generated_summary)
    print("Target Summary:", target_summary)
    print()


In [ ]:
model.train()
learning_rate = 1e-4
num_epochs = 1

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_loss = 0

    for batch in train_loader:
        inputs = {
            "input_ids": torch.stack(batch["input_ids"]).squeeze(dim=1).cpu(),
            "attention_mask": torch.stack(batch["attention_mask"]).squeeze(dim=1).cpu(),
            "decoder_input_ids": torch.stack(batch["decoder_input_ids"]).squeeze(dim=1).cpu(),
            "decoder_attention_mask": torch.stack(batch["decoder_attention_mask"]).squeeze(dim=1).cpu(),
            "labels": torch.stack(batch["labels"]).squeeze(dim=1).cpu(),
        }

        optimizer.zero_grad()

        inputs = {k: v.to(device) for k, v in inputs.items()}  # Move tensors back to GPU

        outputs = model(**inputs)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {avg_loss}")
